In [27]:
import optuna
import joblib
import datetime

In [28]:
from stesml.model_tools import build_train_test_model

In [29]:
def objective(trial):   
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        epochs = 20 # Neglect optmizing # of epochs, allow early stopping to determine # of epochs
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': epochs}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters)
    elif model_type == 'XGBoost':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 50, 10000, log=True)
        parameters = {'n_estimators': n_estimators}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 100, log=True)
        parameters = {'n_estimators': n_estimators}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters)
    
    joblib.dump(study, "../studies/study_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    return result

In [30]:
data_dir = "../data/Sulfur_Models/"
model_type = 'NN' # Options: NN, XGBoost, RandomForest
target = 'Tavg' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = False
study_name = "study_20220630-172431.pkl"

In [32]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

Best trial until now:


ValueError: No trials are completed yet.

In [26]:
study.optimize(objective, n_trials=50)

Epoch 1/20
1438/1438 [==============================] - 3s 2ms/step - loss: 0.0340 - val_loss: 0.0019
Epoch 2/20
1438/1438 [==============================] - 2s 2ms/step - loss: 3.3800e-05 - val_loss: 0.0023
Epoch 3/20
1438/1438 [==============================] - 2s 2ms/step - loss: 2.9536e-05 - val_loss: 0.0026
Split #0, This Result: 2.2319, Average Result: 2.2319
Epoch 1/20
1488/1488 [==============================] - 3s 2ms/step - loss: 0.0351 - val_loss: 0.0030
Epoch 2/20
1488/1488 [==============================] - 2s 2ms/step - loss: 2.5501e-05 - val_loss: 0.0029
Epoch 3/20
1488/1488 [==============================] - 2s 2ms/step - loss: 2.7095e-05 - val_loss: 0.0028
Epoch 4/20
1488/1488 [==============================] - 2s 2ms/step - loss: 3.1185e-05 - val_loss: 0.0028
Epoch 5/20
1488/1488 [==============================] - 2s 2ms/step - loss: 3.0762e-05 - val_loss: 0.0024
Epoch 6/20
1488/1488 [==============================] - 2s 2ms/step - loss: 2.7443e-05 - val_loss: 0.0026


[I 2022-06-30 17:24:31,691] Trial 0 finished with value: 2.465353503701203 and parameters: {'n_layers': 4, 'n_hidden_units': 56, 'batch_size': 1408}. Best is trial 0 with value: 2.465353503701203.


Split #4, This Result: 2.9546, Average Result: 2.4654
Epoch 1/20
10329/10329 [==============================] - 7s 623us/step - loss: 0.0642 - val_loss: 9.3084e-04
Epoch 2/20
10329/10329 [==============================] - 6s 625us/step - loss: 4.2876e-04 - val_loss: 0.0019
Epoch 3/20
  313/10329 [..............................] - ETA: 4s - loss: 2.4558e-04

KeyboardInterrupt: 

In [17]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621